In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torchvision import models
from tqdm import tqdm, trange
import time

In [ ]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((300, 250)),       # Resize images
    transforms.ToTensor(),               # Convert to tensor (0–1 float)
])

# Load datasets
train_dataset = datasets.ImageFolder(
    root="Data/data1a/training",
    transform=transform
)

val_dataset = datasets.ImageFolder(
    root="Data/data1a/validation",
    transform=transform
)

# Create DataLoaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)


# debugging 
# print("Classes:", train_dataset.classes)
# print("Train batches:", len(train_loader))
# print("Val batches:", len(val_loader))


Classes: ['00-damage', '01-whole']
Train batches: 1
Val batches: 1


In [ ]:
class ResNetBinary(nn.Module):
    def __init__(self):
        super().__init__()

        # Load ResNet18 pretrained on ImageNet
        self.model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

        # Get the number of features before the final layer
        num_features = self.model.fc.in_features

        # Replace the final fully connected layer for binary classification (2 outputs)
        self.model.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.model(x)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

def train(model, dataset, epochs):
    optimizer = torch.optim.Adam(model.parameters())
    loss = nn.CrossEntropyLoss()
    dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
    model = model.to(device)
    for epoch in trange(epochs):
        start = time.time()
        for (xs, targets) in tqdm(dataloader):
            xs, targets = xs.to(device), targets.to(device)
            ys = model(xs)
            optimizer.zero_grad()
            l = loss(ys, targets)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                acc = (ys.argmax(axis=1) == targets).sum() / xs.shape[0]
        duration = time.time() - start
        print("[%d] acc = %.2f loss = %.4f in %.2f seconds." % (epoch, acc.item(), l.item(), duration))

Using device: cpu


In [ ]:
model = ResNetBinary()

# training on 4 pictures for testing
train(model, train_dataset, epochs=10)

 10%|█         | 1/10 [00:01<00:10,  1.13s/it]

[0] acc = 0.50 loss = 0.9497 in 1.13 seconds.


 20%|██        | 2/10 [00:02<00:08,  1.08s/it]

[1] acc = 1.00 loss = 0.0137 in 1.05 seconds.


 30%|███       | 3/10 [00:03<00:07,  1.08s/it]

...

[7] acc = 1.00 loss = 0.0001 in 1.10 seconds.


 90%|█████████ | 9/10 [00:09<00:01,  1.10s/it]

[8] acc = 1.00 loss = 0.0001 in 1.08 seconds.


100%|██████████| 10/10 [00:10<00:00,  1.09s/it]

[9] acc = 1.00 loss = 0.0000 in 1.06 seconds.
